In [1]:
# Data extraction tools
import requests
from bs4 import BeautifulSoup

#Data wrangling tools
import pandas as pd
import numpy as np

#DataBase tools
import psycopg2
import psycopg2.extras as extras

# Data Extraction

In [2]:
year_html = None #########

In [3]:
def soup(year):
    
    url = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    
    print(f"Asking for http content from {url}")
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    
    print(f"Filtering by year provided: {year}...")
    global year_html
    year_html = soup.find_all("div", id=f"faq{year}")[0] ##############
    print(f"html generated")

In [4]:
def single_month_links_df(year, month):
    
    months_available = [elem.get_text(strip=True) for elem in year_html.find_all(['b'])]
    
    print(f"Checking month requested {month} vs months available in the html content...")
    if month not in months_available:
        raise ValueError(f"month provided does not match with available months, check spelling {months_available}")
    
    print(f"Month provided is valid. Filtering by month...")
    filter_by_month = year_html.find(string=f"{month}").parent.findNext('ul')
    links = [elem.a['href'] for elem in filter_by_month.find_all('li')]
    description = [elem.a.text for elem in filter_by_month.find_all('li')]
    
    single_month_links = pd.DataFrame({'month': month, 'links': links, 'description': description})
    
    pd.set_option('display.max_colwidth', None)
    
    return single_month_links

def all_months_links_df(year, *month):
    
    links = pd.DataFrame()
    
    for elem in month:
        links = links.append(single_month_links_df(year, elem), ignore_index=True)
 
    print("Generating csv links table")
    
    return links

def csv_to_df(year, taxi_color ,*month):
    
    soup(year) ###
    
    print(f"list of months requested: {month}")
    links_table = all_months_links_df(year, *month)
    links_table = links_table[links_table['description'].str.contains(taxi_color)]['links']
    print(links_table)
    
    for m in links_table:
        print(f"Reading csv {m}")
        single_df = pd.read_csv(m)
        type_object = sintaxi_color_df.select_dtypes(include='object').columns.to_list()
        type_object.pop()
        for typ in type_object:
            single_df[f"{typ}"] = pd.to_datetime(single_df[f"{typ}"], yearfirst=True, format="%Y/%m/%d")
         
        single_df.to_csv(taxi_color + '_' + m[-11:-4] + '.csv')
    
    print("Process completed")

# Data Wrangling

In [5]:
def csv_to_df(year, taxi_color ,month):
    
    soup(year) ###
        
    print(f"Month requested: {month}")
    link = single_month_links_df(year, month)

    link = link[link['description'].str.contains(taxi_color)]['links'][0]
    print(f"Reading csv {link}")
    
    month_df = pd.read_csv(link)
    type_object = month_df.select_dtypes(include='object').columns.to_list()
    type_object.pop()
    for typ in type_object:
        month_df[f"{typ}"] = pd.to_datetime(month_df[f"{typ}"], yearfirst=True, format="%Y/%m/%d")
    
    return month_df

In [ ]:
yellowtaxis_2020_january = csv_to_df(2020, 'Yellow', 'January')
yellowtaxis_2020_january

Asking for http content from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
Filtering by year provided: 2020...
html generated
Month requested: January
Checking month requested January vs months available in the html content...
Month provided is valid. Filtering by month...
Reading csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv


In [ ]:
tuples = [tuple(x) for x in yellowtaxis_2020_january.to_numpy()]
tuples[:2]

In [ ]:
asdafa

# DataBase creation

In [ ]:
#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='postgres', password='Snowdav3', host='127.0.0.1', port= '5432'
)

conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Preparing query to create a database
sql = '''CREATE database taxis''';

#Creating a database
cursor.execute(sql)
print("Database created successfully........")

#Closing the connection
conn.close()

In [ ]:
# DataBase conection

In [ ]:
param_dic = {
    "host"      : "127.0.0.1",
    "database"  : "taxis",
    "user"      : "postgres",
    "password"  : "Snowdav3"
}

In [ ]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print('Connection succesful')
    
    return conn

In [ ]:
conn = connect(param_dic)

# Creating Table in the DataBase

In [ ]:
def creating_table():
    cursor = conn.cursor()
    
    sql ='''CREATE TABLE yellowtaxis(
       VendorID float,
       tpep_pickup_datetime timestamp,
       tpep_dropoff_datetime timestamp,
       passenger_count float,
       trip_distance float,
       RatecodeID float,
       store_and_fwd_flag varchar,
       PULocationID float,
       DOLocationID float,
       payment_type float,
       fare_amount float,
       extra float,
       mta_tax float,
       tip_amount float,
       tolls_amount float,
       improvement_surcharge float,
       total_amount float,
       congestion_surcharge float
    );'''
    
    cursor.execute(sql)
    print("Table created")
    conn.commit()
    cursor.close()
    print("Cursor object closed")
    conn.rollback()
    conn.close()
    print("conn closed")

In [ ]:
creating_table

# DataBase insertion

In [ ]:
def execute_batch(conn, df, table, page_size=100):
    """
    Using psycopg2.extras.execute_batch() to insert the dataframe
    """
    print("Converting DataFrame rows into tuples")
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        print(f"Inserting values into the {table} table")
        extras.execute_batch(cursor, query, tuples, page_size)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print(f"Batch executed succesfully, all dataframe values inserted into the {table} table at {cursor.connection}")
    cursor.close()
    
execute_batch(conn, prueba, 'yellowtaxis') #######

In [ ]:
def execute_multiple_batch(year: int, taxi_color: str, *months, conn, df, table, page_size=100):
    pass